# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **weather phenomena** (see below) for the region of **None, None, Netherlands**, or **Netherlands** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **None, None, Netherlands** to Ann Arbor, USA. In that case at least one source file must be about **None, None, Netherlands**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **None, None, Netherlands** and **weather phenomena**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **weather phenomena**?  For this category you might want to consider seasonal changes, natural disasters, or historical trends.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

weatherDataRaw = pd.read_fwf('KNMI_20191231.txt', header=0, skiprows=11)

In [ ]:
weatherData = weatherDataRaw.drop([0], axis=0).drop(['#'], axis=1)
weatherData.rename(columns={'STN,YYYYMMDD,': 'YYYYMMDD','TN,': 'TN', 'TX,': 'TX'}, inplace=True)
weatherData['STN'] = weatherData['YYYYMMDD'].str.replace(',', '').str[:3]
weatherData['YYYYMMDD'] = pd.to_datetime(weatherData['YYYYMMDD'].str.replace(',', '').str[3:])
weatherData['TN'] = weatherData['TN'].str.replace(',', '')
weatherData['TN'] = pd.to_numeric(weatherData['TN'], errors='coerce')
weatherData['TN'] = weatherData['TN'] * 0.1
weatherData['TX'] = weatherData['TX'] * 0.1

maxTemperature = weatherData[(weatherData['TX'] > 25)][['STN', 'YYYYMMDD', 'TX']].reset_index(drop=True)
minTemperature = weatherData[(weatherData['TN'] < 0)][['STN', 'YYYYMMDD', 'TN']].reset_index(drop=True)

def wave_identifier(df):
    wave = []
    i = 1
    j = 0
    k = 0
    for l in df.index:
        try:
            if (df['YYYYMMDD'][l] - df['YYYYMMDD'][l+1]) == pd.Timedelta('-1 days'):
                if i == 1:
                    j = l
                i += 1
                try:
                    if df['TX'][l] > 30:
                        k += 1
                except:
                    if df['TN'][l] < -10:
                        k += 1
            elif (df['YYYYMMDD'][l] - df['YYYYMMDD'][l+1]) != pd.Timedelta('-1 days'):
                if (i >= 5) and (k >= 3):
                    wave.append((j, l+1))
                    i = 1
                    j = 0
                    k = 0
                else:
                    i = 1
                    j = 0
                    k = 0
        except KeyError:
            pass
    
    return wave

In [ ]:
heatWaveIndices = wave_identifier(maxTemperature)
heatWaves = pd.DataFrame(columns=['Station', 'Mean Temperature', 'Duration', 'Start date', 'End date', 'Year'])

station = []
meanTemperature = []
duration = []
startDate = []
endDate = []
year = []
for i in heatWaveIndices:
    station.append(maxTemperature['STN'][i[0]])
    meanTemperature.append(maxTemperature[i[0]:i[1]]['TX'].mean())
    duration.append(i[1] - i[0])
    startDate.append(str(maxTemperature['YYYYMMDD'][i[0]])[:-9])
    endDate.append(str(maxTemperature['YYYYMMDD'][i[1]-1])[:-9])
    year.append(int(str(maxTemperature['YYYYMMDD'][i[0]])[:-15]))

heatWaves['Station'] = station
heatWaves['Mean Temperature'] = meanTemperature
heatWaves['Duration'] = duration
heatWaves['Start date'] = startDate
heatWaves['End date'] = endDate
heatWaves['Year'] = year

coldWaveIndices = wave_identifier(minTemperature)
coldWaves = pd.DataFrame(columns=['Station', 'Mean Temperature', 'Duration', 'Start date', 'End date', 'Year'])

station = []
meanTemperature = []
duration = []
startDate = []
endDate = []
year = []
for i in coldWaveIndices:
    station.append(minTemperature['STN'][i[0]])
    meanTemperature.append(minTemperature[i[0]:i[1]]['TN'].mean())
    duration.append(i[1] - i[0])
    startDate.append(str(minTemperature['YYYYMMDD'][i[0]])[:-9])
    endDate.append(str(minTemperature['YYYYMMDD'][i[1]-1])[:-9])
    year.append(int(str(minTemperature['YYYYMMDD'][i[0]])[:-15]))

coldWaves['Station'] = station
coldWaves['Mean Temperature'] = meanTemperature
coldWaves['Duration'] = duration
coldWaves['Start date'] = startDate
coldWaves['End date'] = endDate
coldWaves['Year'] = year

plt.figure(figsize=(12,8))

plt.subplot(121)
plt.scatter(x=heatWaves['Duration'], y=heatWaves['Mean Temperature'], s=500, c=heatWaves['Year'], cmap = 'Reds', alpha=0.5)
plt.colorbar().set_label('Year')
plt.xlabel('Duration (number of days)')
plt.ylabel('Mean Temperature ($^\circ C$)')
plt.title('Heat Waves over Netherlands from 1901 to 2019')

plt.subplot(122)
plt.scatter(x=coldWaves['Duration'], y=coldWaves['Mean Temperature'], s=500, c=coldWaves['Year'], cmap = 'Blues', alpha=0.5)
plt.colorbar().set_label('Year')
plt.xlabel('Duration (number of days)')
plt.ylabel('Mean Temperature ($^\circ C$)')
plt.title('Cold Waves over Netherlands from 1901 to 2019')

plt.tight_layout(pad=3.5)